In [1]:
import os
import sys
sys.path.append("/mnt/code/")

In [2]:
import torchmetrics as tm

In [3]:
import os
import tempfile
import time

import mlflow
from ray import air, tune
from ray.air import session
from domino_mlflow_utils.mlflow_callback import *

2024-03-04 10:00:22,861	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-03-04 10:00:23,161	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def train_function(config):
    width, height = config["width"], config["height"]

    for step in range(config.get("steps", 100)):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back to Tune.
        session.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)

In [5]:
def tune_with_callback(mlflow_tracking_uri, experiment_name,finish_fast=False):
    cb = MyMLflowLoggerCallback(
                    tracking_uri=mlflow_tracking_uri,
                    experiment_name=experiment_name,
                    save_artifact=True,
                )
    tuner = tune.Tuner(
        train_function,
        tune_config=tune.TuneConfig(num_samples=5),        
        run_config=air.RunConfig(
            name="mlflow",
            callbacks=[
                cb
            ],
        ),
        param_space={
            "width": tune.randint(10, 100),
            "height": tune.randint(0, 100),
            "steps": 5 if finish_fast else 100,
        },
    )
    results = tuner.fit()
    cb.log_end_parent_run()

In [6]:
import mlflow
import os
import ray
import logging
from domino_mlflow_utils.mlflow_callback import *

temp_dir='/tmp'

if not ray.is_initialized():
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    address=f"ray://{service_host}:{service_port}"
    #temp_dir='/mnt/data//{}/'.format(os.environ['DOMINO_PROJECT_NAME']) #set to a dataset
    ray.init(address=address, _temp_dir=temp_dir,runtime_env={"py_modules": ['/mnt/code/domino_mlflow_utils']})

print('Ray Initializied')
print(f'Ray Host={service_host} and Ray Port={service_port}')

experiment_name = 'RAYv7'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
tune_with_callback(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)

#tune_with_setup(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)
df = mlflow.search_runs(
        [mlflow.get_experiment_by_name(experiment_name).experiment_id]
    )
print(df)


(TunerInternal pid=1010) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
(TunerInternal pid=1010) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(TunerInternal pid=1010) ╭────────────────────────────────────────────────────────╮
(TunerInternal pid=1010) │ Configuration for experiment     mlflow                │
(TunerInternal pid=1010) ├────────────────────────────────────────────────────────┤
(TunerInternal pid=1010) │ Search algorithm                 BasicVariantGenerator │
(TunerInternal pid=1010) │ Scheduler                        FIFOScheduler         │
(TunerInternal pid=1010) │ Number of trials                 5                     │
(TunerInternal pid=1010) ╰────────────────────────────────────────────────────────╯
(TunerInternal pid=1010) 
(TunerInternal pid=1010) View detailed results here: /home/ray/ray_results/mlflow
(TunerInternal pid=1010) To visualize your results with TensorBoard, run: `tensorboard --logdir /home/ray/ray_results/mlflow`
(TunerInternal pid=1010) 
(TunerInternal pid=1010) Trial status: 5 PENDING
(TunerInternal pid=1010) Current time: 2024-03-04 02:00:39. Total running time: 0s
(TunerInternal pid=1